# Reddit Depression Classifier


In [1]:
!pip install happiestfuntokenizing

import pickle
import numpy as np
import pandas as pd
import os
from sklearn.model_selectiotn import cross_validate, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from happiestfuntokenizing.happiestfuntokenizing import Tokenizer
from collections import Countero


from google.colab import drive
drive.mount('/content/drive')

# FILEPATH = 'drive/MyDrive/data/'
FILEPATH = 'drive/MyDrive/Brown Stuff/Computational Linguistics/Assignments/Final Project/student.pkl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [2]:
def load(depression_subreddits, lda):
  """
    Function that loads the dataset with all depression related posts and the control posts. It also
    tokenizes and does stop word removal (but only for lda).

    Parameters
    ----------
    depression_subreddits : list[str]
        the list of all depression related subreddits
    lda : bool
        a boolean to determine if the dataset is needed for lda (ensures tokenizing and stop word removal only done for lda)

    Returns
    -------
    a dataframe with all post's (including the control and all symptoms) text (under post) and subreddit (under symptom)
  """

  CACHE_PATH = f"{FILEPATH}_{'lda' if lda else 'roBERTa'}_processed.pickle"

  #if the cache_path exists that means open the contents of the pickle file
  if os.path.exists(CACHE_PATH):
      datasets = pickle.load(open(CACHE_PATH, "rb"))
      return datasets
  #else dump the posts into a pickle file
  elif lda:
  #generate the original dataset and tokenize
      datasets = dataset_generation(depression_subreddits)

      #only preprocesses for lda
      if lda:
        tokenized_posts_dict = {
          symptom: [tokenize(post) for post in posts]
          for symptom, posts in datasets.items()
        }

        #generate stop_words
        stop_words_data = stop_words(tokenized_posts_dict)
        stop_words_data = stop_words_data.get("Control")

        stop_words_list = []
        for posts in stop_words_data:
          stop_words_list.append(posts[0])

        #remove stop words from symptom
        for symptoms, posts in tokenized_posts_dict.items():
          for index, post in enumerate(posts):
            stop_word_post = []
            for word in post:
              if word not in stop_words_list:
                stop_word_post.append(word)
              else:
                stop_word_post.append("<OOV>")
            tokenized_posts_dict[symptoms][index] = stop_word_post

      #converts symptom dictionary into a dataframe
      symptom_data = []
      for symptom, posts in tokenized_posts_dict.items():
          symptom_data.extend([{
              "symptom": symptom,
              "post": post
          } for post in posts])

      symptom_df = pd.DataFrame(symptom_data)


      # Save the combined data into a pickle file
      pickle.dump(symptom_df, open(CACHE_PATH, "wb"))
      return symptom_df

  else:
    # for non-lda (roBERTa)
    datasets = dataset_generation(depression_subreddits)

    #skip preprocess and load straight into dataframe
    symptom_data = []

    for symptom, posts in datasets.items():
        symptom_data.extend([{
            "symptom": symptom,
            "post": post
        } for post in posts])

    symptom_df = pd.DataFrame(symptom_data)


    # Save the combined data into a pickle file
    pickle.dump(symptom_df, open(CACHE_PATH, "wb"))

    return symptom_df


In [3]:
def dataset_generation(depression_subreddits):
  """
    Function that generates the datasets, finding all posts that correlate to a depression related subreddit or meet control requirements

    Parameters
    ----------
    depression_subreddits : list[str]
        the list of all depression related subreddits

    Returns
    -------
    returns a dictionary with the key being the specific depression related subreddit or 'Control' and the value being a list of the actual text of every
    post related to that subreddit/'Control'
  """

  #open initial file and pull information related to each category
  with open(FILEPATH, 'rb') as f:
    df = pd.read_pickle(f)
    reddit_posts = df['text']
    reddit_authors = df['author']
    subreddit_cats = df['subreddit']
    post_utc_times = df['created_utc']
    post_dates = df['date']

  control_dataset = []
  symptom_dataset_dict = {}

  #input author's name, returns UTC of oldest post
  author_oldest_post_dict = {}

  #input author's name, get UTC of every post
  author_to_post_UTC_dict = {}

  #creating empty lists for each depression category
  for symptoms in depression_subreddits:
    symptom_dataset_dict[symptoms] = []
  symptom_dataset_dict["Control"] = []

  #appends to symptom dataset
  #for every post
  for index in range(len(reddit_posts)):
    #checks if the specific post has a category and if the category is a depression subreddit
    if index in subreddit_cats.index:
      if subreddit_cats[index] in depression_subreddits:
        #if the author is already a key in the dictionary, just append the utc for all the posts, else also add the key
        if reddit_authors[index] in author_to_post_UTC_dict:
          author_to_post_UTC_dict[reddit_authors[index]].append(post_utc_times[index])
        else:
          author_to_post_UTC_dict[reddit_authors[index]] = [post_utc_times[index]]

        #add to symptom subreddit
        current_symptom = subreddit_cats[index]
        symptom_dataset_dict[current_symptom].append(reddit_posts[index])

  i = 0
  #appends to control dataset
  for index in range(len(reddit_posts)):
    #verify that the specific category exists and is not a depression topic
    if index in subreddit_cats.index:
      if subreddit_cats[index] not in depression_subreddits:
        i+=1
        # verify at least six months have passed between control and depression post
        if six_months_passed(post_utc_times[index], reddit_authors[index], author_oldest_post_dict, author_to_post_UTC_dict):
          #append post to control dataset
          symptom_dataset_dict["Control"].append(reddit_posts[index])
          # control_dataset.append(reddit_posts[index])


  # return (control_dataset, symptom_dataset_dict)
  return symptom_dataset_dict

def six_months_passed(utc: int, author_name, author_oldest_post_dict, author_to_post_UTC_dict):
  """
    this helper method for control dataset generation returns a boolean depending on whether the control post is 180 days
    older than the mental health one

    Parameters
    ----------
    utc : int
      the utc of the control post candidate

    author_name : str
      the name of the author

    author_oldest_post_dict : dict
      key is the author's name, value is the utc of the oldest post

    author_to_post_UTC_dict : dict
      key is the author's name, value is the utc of every post

    Returns
    -------
    returns a boolean telling you whether the control post is 180 days older than the mental health one
  """
  #use a dictionary to find the author's oldest post UTC
  oldest_author_post = author_oldest_post_dict.get(author_name)

  #will only update dictionary to find an author's new post if its a new author
  if not oldest_author_post:
    oldest_author_post = find_oldest_depression_post(author_name, author_to_post_UTC_dict)
    author_oldest_post_dict[author_name] = oldest_author_post

  #tells you if 180 days have passed between the mental health and non-mental health post
  if oldest_author_post != None:
    if oldest_author_post:
      if oldest_author_post - utc >= 15552000:
        return True
  return False

#this method finds the oldest depression post for a specific author
def find_oldest_depression_post(name, author_to_post_UTC_dict):
  """
    helper method for six_months_passed() method that finds the oldest depression post UTC

  """
  #gets all the UTC times of every post a specific author made
  UTC_times = author_to_post_UTC_dict.get(name)

  #sorts every post to find the lowest UTC (oldest depression post) and returns it (or None if an author has 0 depression posts)
  if UTC_times:
    UTC_times.sort()
    return UTC_times[0]
  return None

In [4]:
# List of depression subreddits
depression_subreddits = ["Anger",
    "anhedonia", "DeadBedrooms",
    "Anxiety", "AnxietyDepression", "HealthAnxiety", "PanicAttack",
    "DecisionMaking", "shouldi",
    "bingeeating", "BingeEatingDisorder", "EatingDisorders", "eating_disorders", "EDAnonymous",
    "chronicfatigue", "Fatigue",
    "ForeverAlone", "lonely",
    "cry", "grief", "sad", "Sadness",
    "AvPD", "SelfHate", "selfhelp", "socialanxiety", "whatsbotheringyou",
    "insomnia", "sleep",
    "cfs", "ChronicPain", "Constipation", "EssentialTremor", "headaches", "ibs", "tinnitus",
    "AdultSelfHarm", "selfharm", "SuicideWatch",
    "Guilt", "Pessimism", "selfhelp", "whatsbotheringyou"
 ]

In [5]:
import string

def tokenize(unprocessed_posts):
  """
    input the posts and tokenizes the post for lda dataset

  """
  # tokenizes posts
  tokenizer = Tokenizer()
  tokens = tokenizer.tokenize(unprocessed_posts)

  # list of all unqiue punctuation tokens
  punctuation = set(string.punctuation)

  # Filter out the punctuation tokens
  filtered_tokens = [token for token in tokens if token not in punctuation]
  return filtered_tokens


In [6]:
def stop_words(datasets):
  """
    finds the 100 most common stop words for each symptom and control (technically only control is neccessary)
    and puts them into a dict

  """
  # get 100 most common words in control
  most_common_word_dict = {}

  # for key, values in symptoms_dataset_dict.items():
  for key, values in datasets.items():
    # Iterate through the dictionary and update the Counter with words from each list
    word_counter = Counter()
    for words in values:
      word_counter.update(words)

    # Find the 100 most common words across all keys
    most_common_words = word_counter.most_common(100)

    most_common_word_dict[key] = most_common_words

  return most_common_word_dict


In [7]:
# key to map 40+ subreddits to 13 symptoms
symptom_mapping = {
    'Control': ['Control'],
    'Anger': ['Anger'],
    'Anhedonia': ['Anhedonia', 'DeadBedrooms'],
    'Anxiety': ['Anxiety', 'AnxietyDepression', 'HealthAnxiety', 'PanicAttack'],
    'Concentration deficit': ['DecisionMaking', 'shouldi'],
    'Disordered eating': ['bingeeating', 'BingeEatingDisorder', 'EatingDisorders', 'eating_disorders', 'EDAnonymous'],
    'Fatigue': ['chronicfatigue', 'Fatigue'],
    'Loneliness': ['ForeverAlone', 'lonely'],
    'Sad mood': ['cry', 'grief', 'sad', 'Sadness'],
    'Self-loathing': ['AvPD', 'SelfHate', 'selfhelp', 'socialanxiety', 'whatsbotheringyou'],
    'Sleep problem': ['insomnia', 'sleep'],
    'Somatic complaint': ['cfs', 'ChronicPain', 'Constipation', 'EssentialTremor', 'headaches', 'ibs', 'tinnitus'],
    'Suicidal thought and attempts': ['AdultSelfHarm', 'selfharm', 'SuicideWatch'],
    'Worthlessness': ['Guilt', 'Pessimism', 'selfhelp', 'whatsbotheringyou']
}

# actually maps subreddits to symptoms
def map_subreddit_to_symptom(subreddit, symptom_mapping):
    # if a subreddit maps to the symptom, return the symptom
    for symptom, subreddits in symptom_mapping.items():
        if subreddit in subreddits:
            return symptom
    # Return 'Other' if subreddit doesn't correlate to symptom
    return 'Other'

## RoBERTa Embeddings

In [8]:
# get the roberta dataset (no tokenizing or stop word removal)
roberta_dataset = load(depression_subreddits, False)

In [9]:
def get_embeddings(input_text):
    """
    helper method for run_roberta() that allows you to input your dataset and it gives the averaged word embeddings for all posts
    in control and the symptom datasets

    """
    # intialize tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
    model = RobertaModel.from_pretrained('distilroberta-base')

    # enables model to use GPU and prevents model from crashing
    device = torch.device("cuda")
    model.to(device)
    model.eval()

    # average embeddings for each posts (list)
    avg_embeddings = []

    # for every post
    for post in input_text['post'].tolist():

      # tokenize the post then send to GPU
      encoded_posts = tokenizer(post, padding=True, truncation=True, max_length=512, return_tensors="pt")
      encoded_posts.to(device)

      # input tokenized words into model
      with torch.no_grad():
        outputs = model(**encoded_posts)

      # extract word embeddings from the last hidden state and average the word embeddings to get an average post embedding
      last_hidden_state = outputs.last_hidden_state
      avg_embedding = last_hidden_state.mean(dim=1).cpu().numpy()
      avg_embeddings.append(avg_embedding)

    # cast this list into a numpy array and return
    numpy_embeddings = np.array(avg_embeddings)
    return numpy_embeddings

In [17]:
import torch
from transformers import RobertaTokenizer, RobertaModel

def run_roberta():
  """
    runs roberta, getting the embeddings from the helper method above, and putting the embeddings in X and actual symptoms in y and
    running with main to output accuracies by symptom

  """
  # prevents the need to get embeddings each time with a pickle file
  CACHE_PATH = f"{FILEPATH}_embeddings.pickle"

  if os.path.exists(CACHE_PATH):
    avg_embeddings = pickle.load(open(CACHE_PATH, "rb"))
  else:
    embeddings = get_embeddings(roberta_dataset)
    pickle.dump(embeddings, open(CACHE_PATH, "wb"))
    avg_embeddings = embeddings

  # maps posts by subreddit to the correct symptom (ignore the 'symptom' designation that's acutaally the subreddit col)
  roberta_dataset['grouped_symptom'] = roberta_dataset['symptom'].apply(lambda x: map_subreddit_to_symptom(x, symptom_mapping))

  # get the list of all symptoms
  symptom_list = roberta_dataset['grouped_symptom'].tolist()

  # Iterate through the symptom list and get a dictionary with the keys being each specific symptom + control and the values returned being
  # the index of each post corresponding to the symptom/control
  symptom_groups = {}

  for index, post_symptom in enumerate(symptom_list):
      symptom_groups.setdefault(post_symptom, []).append(index)

  print("roBERTa Outputs (accuracy is in decimal form): ")
  desired_symptoms = ['Worthlessness', 'Anger', 'Anhedonia', 'Anxiety', 'Disordered eating', 'Loneliness', 'Sad mood', 'Self-loathing', 'Sleep problem', 'Somatic complaint']

  # for each symptom you want
  for symptom in desired_symptoms:

    X = []
    y = []

    # combine the specific symptom and control posts
    combined_list = symptom_groups.get(symptom) + symptom_groups.get("Control")

    # for each post, add the speicific symptom as y and the word embedding as X
    for index in combined_list:
      y.append(symptom_list[index])
      X.append(avg_embeddings[index])

    # convert y into a binary with 0 meaning Control and 1 meaning the specific symptom
    new_y = []
    for y_symptom in y:
      if y_symptom == 'Control':
        new_y.append(0)
      else:
        new_y.append(1)

    # case X and y into numpy arrays
    X = np.array(X)
    X = X.squeeze(axis=1)
    y = np.array(new_y)

    # run main to evaluate and print each symptom
    accuracy_by_symptom = main(X, y)

    print(symptom + ": " + str(accuracy_by_symptom))


## Reddit Topics with LDA


In [14]:
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore

def run_lda_on_all(datasets):
    """
    simply just runs lda on all posts in the dataset, returning the model, dictionary (idx2word), and corpus (combo of topics per doc)

    """
    # make a list of all posts and check they aren't blank
    all_posts = datasets['post'].tolist()

    valid_posts = []
    for post in all_posts:
      if post and len(post) > 0:
        valid_posts.append(post)

    # Create idx2word for all posts
    idx2word = Dictionary(valid_posts)

    # Create corpus (bag of words) for each post
    corpus = []
    for post in valid_posts:
      corpus.append(idx2word.doc2bow(post))

    # run lda
    lda_model = LdaMulticore(corpus=corpus, id2word=idx2word, num_topics=200, passes=2)

    # Return the LDA model, dictionary, and corpus
    lda_outputs = {'model': lda_model,'dictionary': idx2word,'corpus': corpus}

    return lda_outputs

In [15]:
def run_lda():
  """
    runs the actual lda, getting the lda outputs from the helper method above, and converting it into topic distributions you put into X and symptoms
    you put into y for the main function, then returning the accuracy of each symptom

  """
  # load the lda dataset (including tokenizing and stop word removal)
  lda_datasets = load(depression_subreddits, True)

  # assigns the subreddits to the symptoms (ignore the 'symptom' designation that's acutaally the subreddit col)
  lda_datasets['grouped_symptom'] = lda_datasets['symptom'].apply(lambda x: map_subreddit_to_symptom(x, symptom_mapping))

  #prevents LDA from having to run every time with a pickle file
  CACHE_PATH = f"{FILEPATH}_lda_results.pickle"

  if os.path.exists(CACHE_PATH):
    lda_results = pickle.load(open(CACHE_PATH, "rb"))
  #else dump the posts into a pickle file
  else:
    lda_results = run_lda_on_all(lda_datasets)
    pickle.dump(lda_results, open(CACHE_PATH, "wb"))
    lda_results = lda_results

  print("LDA Outputs: ")
  desired_symptoms = ['Worthlessness', 'Anger', 'Anhedonia', 'Anxiety', 'Disordered eating', 'Loneliness', 'Sad mood', 'Self-loathing', 'Sleep problem', 'Somatic complaint']
  #shuffle dataset to improve model accuracy
  lda_datasets = lda_datasets.sample(frac=1, random_state=42).reset_index(drop=True)

  # run lda for each symptom
  for symptom in desired_symptoms:

    #filter datasets just for specific symptom and control
    control_data = lda_datasets[lda_datasets['grouped_symptom'] == 'Control']
    symptom_data = lda_datasets[lda_datasets['grouped_symptom'] == symptom]

    # Combine the control and specific symptom data into one dataset and shuffle
    lda_datasets_filtered = pd.concat([symptom_data, control_data])
    lda_datasets_filtered = lda_datasets_filtered.sample(frac=1, random_state=42).reset_index(drop=True)

    # creates a new corpus only with posts from the filtered dataset (the specific symptom + control)
    filtered_corpus = []
    all_filtered_posts = lda_datasets_filtered['post'].tolist()

    for post in all_filtered_posts:
        filtered_corpus.append(lda_results.get("dictionary").doc2bow(post))


    # get topic distributions and convert it into a numpy array which is a matrix of topic vectors
    topic_distributions = [lda_results.get("model").get_document_topics(doc) for doc in filtered_corpus]
    X = convert_to_topic_matrix(topic_distributions, num_topics=200)

    # prepare answer key to evaluate model (0 is "Control" and 1 is specific symptoms)
    y = lda_datasets_filtered['grouped_symptom'].apply(lambda x: 0 if x == 'Control' else 1).values

    # run main to get specific symptoms and print
    accuracy_by_symptom = main(X, y)
    print(symptom + ": " + str(accuracy_by_symptom))

def convert_to_topic_matrix(topic_distributions, num_topics=200):

    """
    For each post in the filtered corpus, create a 200 length zero vector and assign topic probabilities to each topic, return a numpy
    array with the topic vector for each doc
    """

    padded_topic_matrix = []

    # for each post, create an all 0 vector of length 200 topics (total number of topics)
    for post in topic_distributions:
        all_zero_vector = [0] * num_topics
        # fill in the topic probabilities that appear in the doc
        for topic, topic_prob in post:
            all_zero_vector[topic] = topic_prob
        padded_topic_matrix.append(all_zero_vector)
    #return a numpy array that has the probability of each topic for each post
    return np.array(padded_topic_matrix)


## Main

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
import warnings


def main(X, y):
    """
    Runs 5-fold cross-validation with Random Forest to evaluate your LDA performance.
    """
    rf_classifier = RandomForestClassifier()
    cv = KFold(n_splits=5, shuffle=True)

    # stores the models outputs as well as the answer key
    model_predictions = []
    actual_results = []

    # Run cross-validation
    for training, testing in cv.split(X):

        # Split X and y into train and test sets
        X_train = X[training]
        X_test = X[testing]
        y_train = y[training]
        y_test = y[testing]

        # Train classifier and get data
        rf_classifier.fit(X_train, y_train)
        predictions = rf_classifier.predict(X_test)

        # Store the predictions and the answer key
        model_predictions.extend(predictions)
        actual_results.extend(y_test)

    # evaluate accuracy by comparing the predictions to the actual result
    accuracy_by_symptom = roc_auc_score(model_predictions, actual_results)

    # return actual accuracy
    return accuracy_by_symptom

#print the actual outputs by symptom for lda and roberta
run_roberta()
print("\n")
run_lda()



roBERTa Outputs (accuracy is in decimal form): 
Worthlessness: 0.8272385771235817
Anger: 0.9331138258810611
Anhedonia: 0.9055510874886356
Anxiety: 0.9151186024247584
Disordered eating: 0.9263507513419804
Loneliness: 0.8697202356709411
Sad mood: 0.868400837988827
Self-loathing: 0.8785214922278957
Sleep problem: 0.936635594804224
Somatic complaint: 0.8784724103899423


LDA Outputs: 
Worthlessness: 0.6605677023398542
Anger: 0.8195432177128358
Anhedonia: 0.8490962280040778
Anxiety: 0.8355403933539782
Disordered eating: 0.8708661513290473
Loneliness: 0.7637219222764429
Sad mood: 0.7641562535179556
Self-loathing: 0.7716024286073371
Sleep problem: 0.9004115293058883
Somatic complaint: 0.7764582002436976
